In [ ]:
import os
import csv

from sklearn.metrics import f1_score, recall_score


def evaluate_and_save_classification_results(model_name, y_test, y_pred, output_file="classification_results.csv"):
    accuracy = accuracy_score(y_test, y_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)

    # extract features from confusion matrix
    tn, fp, fn, tp = confusion_matrix_result.ravel()

    precision = tp / (tp + fp)

    # calculate F1-score and recall directly
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # prepare data for CSV
    flattened_report = {
        'model_name': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'F1-Score': f1,
        'Recall': recall,
        'True Positives': tp,
        'True Negatives': tn,
        'False Positives': fp,
        'False Negatives': fn,
    }

    # write data to csv
    file_exists = os.path.isfile(output_file)

    with open(output_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=flattened_report.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(flattened_report)

    print(f"Metrics have been saved to {output_file}")


evaluate_and_save_classification_results("Logistic Model", y_test, y_pred)

